<a href="https://colab.research.google.com/github/shahabday/BAMline4CT/blob/main/notebooks/final/Create_HF_dataset_and_kaggle_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction :

we have our jaguars dataset in form of a fifty one dataset.
in order to share this with the community , it is worth it if we upload it not only to fifty one website , but also to the hugging face  .
Later on it is planned to create a kaggle competetion using this dataset .

the dataset lives as of now in our google drive in this format :

* images / raw
* images / cropped body
* images / segmented body
* images / cropped Face
* images / segmented Face

the metadata is stored in a fifty one dataste with

* labels
* train / test / unkown / discarded

the train test splitting is done in another notebook with a selection rule:

1. similar images are found for each label (Jaguar ID)
2. if unique images are less than 5 , that jaguar is labeled as unknown otherwise:
3. clusters are sorted from least populated to most populated
4. 30% of the clusters are chosen for test data :
5. within the chosen clusters for test, one image is randomly selected and the rest are discarded .

(The test train splitting will not be a part of the published dataset)





## Create Hugging Face dataset



```
huggingface_dataset/
├── images/
│   ├── raw/
│   │   ├── 00000.jpg
│   │   ├── 00001.jpg
│   │   └── ...
│   ├── cropped/
│   │   └── ...
│   └── segmented/
│       └── ...
├── metadata_raw.json
├── metadata_cropped.json
├── metadata_segmented.json
└── README.md
```



## Create test set folder for kaggle


# import dataset


In [3]:
!pip install fiftyone
!pip install datasets huggingface_hub


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 11.2 MB/s eta 0:00

In [1]:
from pathlib import Path
import fiftyone as fo
from google.colab import drive
import pandas as pd

In [2]:
# read dataset


# mount shahab
#drive.mount('/content/drive')
# mount davide
#drive.mount('/gdrive')

project_path = {"shahab" : Path("/content/drive/MyDrive/DataScience/Jaguars_Project") ,
             "davide" : Path("/gdrive/MyDrive/DSR/Jaguars_Project/")}

drive.mount("/content/drive/")


user = "shahab"


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
image_dir = project_path[user] / Path("images/cropped_body")
input_dir = project_path[user] / Path("datasets/version_16")

dataset = fo.Dataset.from_dir(
    dataset_dir=str(input_dir),
    dataset_type=fo.types.FiftyOneDataset,
    rel_dir=image_dir,
)

Importing samples...


INFO:fiftyone.utils.data.importers:Importing samples...


 100% |███████████████| 3598/3598 [962.2ms elapsed, 0s remaining, 3.8K samples/s]      


INFO:eta.core.utils: 100% |███████████████| 3598/3598 [962.2ms elapsed, 0s remaining, 3.8K samples/s]      


In [4]:
dataset

Name:        2025.04.10.08.16.07
Media type:  image
Num samples: 3598
Persistent:  False
Tags:        []
Sample fields:
    id:                               fiftyone.core.fields.ObjectIdField
    filepath:                         fiftyone.core.fields.StringField
    tags:                             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:                       fiftyone.core.fields.DateTimeField
    last_modified_at:                 fiftyone.core.fields.DateTimeField
    ground_truth:                     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    prediction:                       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations_body:               fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    bboxes_head:      

In [5]:
# get file path , groundtruth.label, and test_train_split for each entry in dataset
filepath = dataset.values("filepath")
# str(Path(*filepath[0].split("/")[-2:]))  #this makes a relative path like Ousado/P_Ousado_6.jpg
rel_filepath = [ str(Path(*p.split("/")[-2:])) for p in filepath] # get the last two directories
labels = [l.label for l in dataset.values("ground_truth")] # labels
test_train_split = dataset.values("testtrainsplit_cosine_similarity") # {None, 'eliminated', 'test', 'train', 'unknown'}


In [6]:
df = pd.DataFrame({"filepath" : rel_filepath , "label" : labels , "test_train_split" : test_train_split})

In [7]:
df.head()

,filepath,label,test_train_split
0,Ousado/P_Ousado_6.jpg,Ousado,train
1,Ousado/P_Ousado_5.jpg,Ousado,train
2,Ousado/P_Ousado_8.jpg,Ousado,train
3,Ousado/P_Ousado_12.jpg,Ousado,train
4,Ousado/P_Ousado_1.jpg,Ousado,train


In [8]:
# select only the data that is going to be published to the hugging face :
# test, None and eliminated are not included .

df_hf = df[df["test_train_split"].isin(["train", "unknown"])]

In [11]:
df_hf.describe()

,filepath,label,test_train_split
count,3361,3361,3361
unique,3361,47,2
top,Estella/P_Estella_20.jpg,Medrosa,train
freq,1,297,3098


In [10]:
# add unique file numbers in formate : 0000.jpg 0001.jpg
# reindex first
df_hf = df_hf.reset_index()
df_hf["file_number"] = df_hf.index.astype(str).str.zfill(5)

In [15]:
df_hf

,index,filepath,label,test_train_split,file_number
0,0,Ousado/P_Ousado_6.jpg,Ousado,train,00000
1,1,Ousado/P_Ousado_5.jpg,Ousado,train,00001
2,2,Ousado/P_Ousado_8.jpg,Ousado,train,00002
3,3,Ousado/P_Ousado_12.jpg,Ousado,train,00003
4,4,Ousado/P_Ousado_1.jpg,Ousado,train,00004
...,...,...,...,...,...
3356,3593,Estella/P_Estella_29.jpg,Estella,train,03356
3357,3594,Estella/P_Estella_49.jpg,Estella,train,03357
3358,3595,Estella/P_Estella_50.jpg,Estella,train,03358
3359,3596,Estella/P_Estella_23.jpg,Estella,train,03359


In [12]:
# substitute label with test_train_split unknown

df_hf.loc[df_hf["test_train_split"]=="unknown", "label"] = "unknown"

In [14]:

df_hf.describe()

,index
count,3361.000000
mean,1798.871169
std,1027.156445
min,0.000000
25%,912.000000
50%,1811.000000
75%,2681.000000
max,3597.000000


In [18]:
"""
[
  {
    "filename": "00000.jpg",
    "label": "Jaguar_01",
    }, ...
  ]
"""
# create a list of dictionary for each record,
# that has a filename key , file_number.jpg
# and
# label : label

metadata = []


In [46]:
# Now Create a copy function based on the dataframe :

# copy files from origin to their destination

# origin is : base folder + filepath  -> destination : base folder / filenumber.jpg

# then create the metadata json for the copy :

# import loging
import logging
from pathlib import Path
import pandas as pd

class create_HF_dataset :
  '''
  This class creates a HF dataset based on a dataframe.

  The dataframe should have the following columns :
  - filepath
  - label
  - test_train_split *optional
  - file_number *optional

  The class should get the following parameters :
  - base_folder
  - destination_folder

  This class then creates a hugging face dataset in several steps:

  1. filtering :
    the dataframe is fed to the class either filtered, or user can filter the dataframe themselves.
    This part is not implemented yet due the project limited time.

  3. create ordered numbers for all files in the dataframe to rename them.

  4. creates a dictionary of metadata for hugging face :

  [
  {
    "filename": "00000.jpg",
    "label": "Jaguar_01",
    }, ...
  ]

  5. creates a dictionary of files that need to be copied :
    [origin file path ]: [destination file path]



  '''

  def __init__(self, dataframe,base_folder, destination_folder):
    self.dataframe = dataframe
    self.base_folder = base_folder
    self.destination_folder = destination_folder




  def filter_dataframe():
    raise NotImplementedError("This part is not implemented yet")

  def create_file_numbers(self, override = False):

    if override == True :
      if "file_number" not in self.dataframe.columns :
        self.dataframe = self.dataframe.reset_index()
        self.dataframe["file_number"] = self.dataframe.index.astype(str).str.zfill(5)
        logging.info("File numbers created")
    else :
      logging.info("File numbers already exist")



  def _create_metadata(self, df, label_col="label", file_col="file_number", extra_cols=None, file_ext=".jpg"):
    """
    Creates metadata from a dataframe dynamically.

    Parameters:
    - df: pandas.DataFrame
    - label_col: str, column name to use as label
    - file_col: str, column name used to construct the filename
    - extra_cols: list of str, other columns to include in the metadata
    - file_ext: str, file extension to add to the filename (default ".jpg")
    """
    if extra_cols is None:
        extra_cols = []

    def make_metadata_dict(row):
        meta = {
            "filename": f"{row[file_col]}{file_ext}",
            "label": row[label_col]
        }
        for col in extra_cols:
            meta[col] = row[col]
        return meta

    self.metadata = df.apply(make_metadata_dict, axis=1).tolist()
    logging.info("Metadata created with columns: %s", ["filename", "label"] + extra_cols)
    return self.metadata


  def _create_copy_dict(self):
    """
    creates a dictionary of files that need to be copied :
    [origin file path ]: [destination file path].jpg

    Note that the filename will be changed to the file_number.jpg

    """
    self.copy_dict = {}
    for index, row in self.dataframe.iterrows():
      origin_path = self.base_folder / Path(row["filepath"])
      destination_path = self.destination_folder / Path(row["file_number"] + ".jpg")
      self.copy_dict[origin_path] = destination_path # creates the dictionary entry
    logging.info("Copy dictionary created")
    return self.copy_dict

  def create_dataset (self):
    self.create_file_numbers()
    self._create_metadata(self.dataframe)
    self._create_copy_dict()
    return self.metadata, self.copy_dict




# time to copy some data to Hugging Face

In [47]:
import shutil
import os

base_path = project_path[user] / Path("images/cropped_body")
huggingFace = Path("huggingFace/cropped_body")

copy_dict = dataset.copy_dict
dataset =create_HF_dataset(df_hf , base_path,huggingFace )
metadata , path_dict = dataset.create_dataset()


In [ ]:
path_dic

In [28]:
from tqdm import tqdm
import shutil
import os

# Copy files with progress bar
for src, dst in tqdm(copy_dict.items(), desc="Copying files", total=len(copy_dict)):
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copy(src, dst)

In [31]:
metadata[0]

{'filename': '00000.jpg', 'label': 'Ousado'}

In [38]:
from datasets import Dataset, DatasetDict, Features, ClassLabel, Value
import json
import pandas as pd
import datasets

# Add full image path
for sample in metadata:
    sample["image"] = os.path.join("images/cropped_body", sample["filename"])

# Convert to pandas then to Dataset
df = pd.DataFrame(metadata)
dataset = Dataset.from_pandas(df)

# Optionally cast columns
dataset = dataset.cast_column("image", datasets.Image())

# Test
print(dataset[0]["image"])


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2680x2152 at 0x7945DCCD5290>


In [ ]:
dataset[0]["image"]

In [40]:
from huggingface_hub import login

from google.colab import userdata
token = userdata.get('HF_token')

login(token=token)  # Or set HF_TOKEN env var

In [41]:
dataset.push_to_hub("shahabdaiani/jaguaridentification")

Uploading the dataset shards:   0%|          | 0/5 [00:00<?, ?it/s]

Map:   0%|          | 0/673 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/672 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/672 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shahabdaiani/jaguaridentification/commit/2c3ae218fc7d5b7f036ca29238c94af06609231f', commit_message='Upload dataset', commit_description='', oid='2c3ae218fc7d5b7f036ca29238c94af06609231f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/shahabdaiani/jaguaridentification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='shahabdaiani/jaguaridentification'), pr_revision=None, pr_num=None)

# copy all data to hugging face

In [53]:
!rm -rf images

In [ ]:
from tqdm import tqdm
folders = ["raw_images" , "cropped_body" , "cropped_head" , "segmented_body"]

for folder in folders :
  base_path = project_path[user] / Path(f"images/{folder}")
  huggingFace = Path(f"images/{folder}")


  dataset =create_HF_dataset(df_hf , base_path,huggingFace )
  metadata , path_dict = dataset.create_dataset()
  copy_dict = dataset.copy_dict

  print (f"Now copying data")
  # Copy files with progress bar
  for src, dst in tqdm(copy_dict.items(), desc="Copying files", total=len(copy_dict)):
      os.makedirs(os.path.dirname(dst), exist_ok=True)
      shutil.copy(src, dst)


In [61]:
folders = ["raw_images" , "cropped_body" , "cropped_head" , "segmented_body"]
dataset_dict = {}



for folder in folders :

  print( f"I am working on {folder}")
  # copy data :


  base_path = project_path[user] / Path(f"images/{folder}")
  huggingFace = Path(f"images/{folder}")


  dataset =create_HF_dataset(df_hf , base_path,huggingFace )
  metadata , path_dict = dataset.create_dataset()
  copy_dict = dataset.copy_dict

  # Add full image paths
  for sample in metadata:
      sample["image"] = os.path.join(f"images/{folder}", sample["filename"])

  # Convert to HuggingFace Dataset
  df = pd.DataFrame(metadata)
  ds = Dataset.from_pandas(df)

  # Cast image column
  ds = ds.cast_column("image", datasets.Image())

  dataset_dict[folder] = ds

from datasets import DatasetDict

final_dataset = DatasetDict(dataset_dict)




I am working on raw_images
I am working on cropped_body
I am working on cropped_head
I am working on segmented_body


In [56]:
from huggingface_hub import login

from google.colab import userdata
token = userdata.get('HF_token')

login(token=token)  # Or set HF_TOKEN env var

In [62]:
final_dataset.push_to_hub("shahabdaiani/jaguaridentification")

Uploading the dataset shards:   0%|          | 0/76 [00:00<?, ?it/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shahabdaiani/jaguaridentification/commit/6f863304e9e6c1b278145d418ab2fb90b3b652ee', commit_message='Upload dataset (part 00001-of-00002)', commit_description='', oid='6f863304e9e6c1b278145d418ab2fb90b3b652ee', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/shahabdaiani/jaguaridentification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='shahabdaiani/jaguaridentification'), pr_revision=None, pr_num=None)

In [72]:
# labels contains all labels in the dataset
# I want to count how many data per label is there in the datset
label_series = pd.Series(labels)

# Step 2: Get the normalized class distribution (i.e., probabilities)
class_probs = label_series.value_counts(normalize=True)

# Step 3: Calculate expected accuracy of a prior-based random classifier
expected_accuracy = (class_probs ** 2).sum()

In [75]:
expected_accuracy

np.float64(0.04599893523701173)